<a href="https://colab.research.google.com/github/Rishit2105/ecommerce-revenue-risk-sql/blob/master/python/profit_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import os
os.listdir("/content")


['.config',
 'olist_order_items_dataset.csv',
 'olist_customers_dataset.csv',
 'olist_orders_dataset.csv',
 'sample_data']

In [15]:
order_items = pd.read_csv("/content/olist_order_items_dataset.csv")
orders = pd.read_csv("/content/olist_orders_dataset.csv")
customers = pd.read_csv("/content/olist_customers_dataset.csv")


In [16]:
# Profit proxy (30% margin assumption)
order_items["est_profit"] = order_items["price"] * order_items["order_item_id"].apply(lambda x: 1) * 0.30
# NOTE: Olist doesn't have a 'quantity' column. Each row = 1 item.

merged = (
    order_items
    .merge(orders, on="order_id", how="left")
    .merge(customers, on="customer_id", how="left")
)


In [17]:
region_profit = (
    merged.groupby("customer_state")[["price", "est_profit"]]
    .sum()
    .reset_index()
    .rename(columns={"price": "revenue"})
)

region_profit["est_margin"] = region_profit["est_profit"] / region_profit["revenue"]

region_profit.sort_values("est_margin").head(10)


,customer_state,revenue,est_profit,est_margin
8,GO,294591.95,88377.585,0.3
21,RR,7829.43,2348.829,0.3
2,AM,22356.84,6707.052,0.3
0,AC,15982.95,4794.885,0.3
4,BA,511349.99,153404.997,0.3
5,CE,227254.71,68176.413,0.3
6,DF,302603.94,90781.182,0.3
1,AL,80314.81,24094.443,0.3
7,ES,275037.31,82511.193,0.3
9,MA,119648.22,35894.466,0.3


In [18]:
region_profit.to_csv("region_margin_risk.csv", index=False)
